The default settings in Bilbac ccincide with the 'aflow' settings, except for Bilbao = hex, aflow = rhomb

In [20]:

import pandas as pd
import numpy as np
from fractions import Fraction
import json
from sympy import parse_expr,symbols
from kim_tools import get_formal_bravais_lattice_from_space_group, get_change_of_basis_matrix_to_conventional_cell_from_formal_bravais_lattice
import os

SYMMETRY_UTIL_DATA_DIR = '../kim_tools/symmetry_util/data'
GENPOS_MATRICES_NO_CENTERING_FILENAME = 'GENPOS_matrices_no_centering.json'
PRIMITIVE_GENPOS_OPS_FILENAME = os.path.join(SYMMETRY_UTIL_DATA_DIR,'primitive_GENPOS_ops.json')
TRANSLATION_COSET_REPRESENTATIVES_FILENAME = 'translation_coset_representatives.json'
PRIMITIVE_SHIFTS_FILENAME = os.path.join(SYMMETRY_UTIL_DATA_DIR,"possible_primitive_shifts.json")

In [ ]:
# Gets the primitive space group operations in the conventional setting. Meaning the basis is conventional but the additional sets of operations generated by adding centering translations are not included.
sg_ops = {}

for i in range(230):
    sgnum = i+1
    frames = pd.read_html(f"https://www.cryst.ehu.es/cgi-bin/cryst/programs/nph-getgen?gnum={sgnum}&what=gp",match="No",encoding='utf8')
    assert len(frames) == 1, "Expected only one table"
    df = frames[0]
    sg_ops[sgnum] = []
    for op_num,op_string in zip(df[0],df[2]):
        if op_num.isnumeric():
           op_array = np.reshape([float(Fraction(elem)) for elem in op_string.split()],(3,4))
           sg_ops[sgnum].append(op_array.tolist())
     
with open(GENPOS_MATRICES_NO_CENTERING_FILENAME,'w') as f:
    json.dump(sg_ops,f)

In [21]:
# Convert the GENPOS augmented matrices to primitive setting. See 1.5.2.3 in ITA
with open(GENPOS_MATRICES_NO_CENTERING_FILENAME) as f:
    sg_ops = json.load(f)

sg_ops_primitive = {}

for i in range(230):
    sgnum = i+1
    sg_ops_primitive[sgnum] = []
    formal_bravais_lattice = get_formal_bravais_lattice_from_space_group(sgnum)        
    for sg_op in sg_ops[str(sgnum)]:
        # separate matrix and column into keys, useful anyway for accessing
        # old (unprimed) basis: conventional
        # new (primed) basis: primitive
        # Therefore, the below function gets us Q=P^-1 for eqs 1.5.2.12-13
        Qmat = get_change_of_basis_matrix_to_conventional_cell_from_formal_bravais_lattice(formal_bravais_lattice)
        Pmat = np.linalg.inv(Qmat)
        sg_op_mat = np.asarray(sg_op)
        Wmat = sg_op_mat[:,0:3]
        w = sg_op_mat[:,3]
        Wmat_prime = Qmat@Wmat@Pmat
        w_prime = Qmat@w
        sg_ops_primitive[sgnum].append({
            'W': Wmat_prime.tolist(),
            'w': w_prime.tolist()
        })

with open(PRIMITIVE_GENPOS_OPS_FILENAME,'w') as f:
    json.dump(sg_ops_primitive,f)

In [ ]:
# Scrape the unique translation vectors found in the coset representatives of the space group in its normalizer.

x,y,z,r,s,t = symbols("x y z r s t")

translation_parts={}

for i in range(230):
    sgnum = i+1
    if sgnum > 228: # high-symmetry SGs that coincide with their normalizer
        translation_parts[sgnum] = [(0.0, 0.0, 0.0)]
    else:    
        unique_translations = set()
        data = pd.read_html(f"https://www.cryst.ehu.es/cgi-bin/cryst/programs/nph-normsets?from=wycksets&gnum={sgnum}",match="Coset Representative",encoding='utf8')
        for frame in data:
            for datum in frame["Coset Representative"]:
                operations = [parse_expr(coord) for coord in datum.split(',')]
                translations = [float(op.evalf(subs={x: 0, y: 0, z: 0, r: 0, s: 0, t: 0})) for op in operations]
                unique_translations.add(tuple(translations))
        translation_parts[sgnum] = sorted(list(unique_translations))
        
# At least for SG#9 and possibly some other monoclinic SGs, need to manually add translations from the affine normalizer from the NORMALIZER program, as they are not 
# represented in the WYCKSETS program (for SG#15 they are, as a special exception)        
translation_parts[9]+=[
    (0.0, 0.25, 0.0),
    (0.0, 0.5, 0.0),
    (0.0, 0.75, 0.0)
    ]

with open(TRANSLATION_COSET_REPRESENTATIVES_FILENAME,'w') as f:
    json.dump(translation_parts,f)

In [22]:
with open(PRIMITIVE_GENPOS_OPS_FILENAME) as f:
    primitive_GENPOS_ops = json.load(f)
with open(TRANSLATION_COSET_REPRESENTATIVES_FILENAME) as f:
    translation_coset_representatives = json.load(f)
    
possible_primitive_shifts = {}

for i in range(230):
    sgnum = str(i+1)
    possible_primitive_shifts_for_this_sg = []
    primitive_translation_parts_for_this_sg = [GENPOS_op['w'] for GENPOS_op in primitive_GENPOS_ops[sgnum]]
    translation_coset_representatives_for_this_sg = translation_coset_representatives[sgnum]
    formal_bravais_lattice = get_formal_bravais_lattice_from_space_group(sgnum)
    primitivize_matrix = np.transpose(get_change_of_basis_matrix_to_conventional_cell_from_formal_bravais_lattice(formal_bravais_lattice))
    for translation_part_prim in primitive_translation_parts_for_this_sg:
        for coset_rep in translation_coset_representatives_for_this_sg:
            coset_rep_prim = coset_rep@primitivize_matrix
            primitive_shift = translation_part_prim+coset_rep_prim
            primitive_shift = [component%1 for component in primitive_shift]
            already_exists = False
            for existing_prim_shift in possible_primitive_shifts_for_this_sg:
                if np.allclose(primitive_shift,existing_prim_shift):
                    already_exists = True
                    break
            if already_exists:
                continue
            else:
                possible_primitive_shifts_for_this_sg.append(primitive_shift)
    possible_primitive_shifts[sgnum] = possible_primitive_shifts_for_this_sg

with open(PRIMITIVE_SHIFTS_FILENAME,"w") as f:
    json.dump(possible_primitive_shifts,f)